In [1]:
import tensorflow as tf 
from tensorflow.keras import layers 
import numpy as np 
import matplotlib.pyplot as plt 


2025-02-12 16:36:11.077271: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-12 16:36:11.091364: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-12 16:36:11.212629: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-12 16:36:11.359527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739374571.483275   35621 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739374571.51

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
 # Load MNIST dataset 
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data() 
x_train = x_train.astype("float32") / 255.0 
x_train = np.expand_dims(x_train, -1) 
 
latent_dim = 100 

In [3]:
 
# CNN Generator 
def build_generator(): 
    model = tf.keras.Sequential([ 
        layers.Dense(7 * 7 * 256, input_dim=latent_dim), 
        layers.Reshape((7, 7, 256)), 
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", 
activation="relu"), 
        layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", 
activation="relu"), 
        layers.Conv2DTranspose(1, kernel_size=7, activation="tanh", padding="same") 
    ]) 
    return model 
 

In [4]:
# CNN Discriminator 
def build_discriminator(): 
    model = tf.keras.Sequential([ 
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same", input_shape=(28, 28, 
1)), 
        layers.LeakyReLU(alpha=0.2), 
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"), 
        layers.LeakyReLU(alpha=0.2), 
        layers.Flatten(), 
        layers.Dense(1, activation="sigmoid") 
    ]) 
    return model 
 
# GAN Model 
generator = build_generator() 
discriminator = build_discriminator() 

/home/angele/workspace/env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1739375053.538693   35621 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/angele/workspace/env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instea

In [ ]:
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002), 
loss="binary_crossentropy", metrics=["accuracy"]) 
discriminator.trainable = False 
 
gan_input= layers.Input(shape=(latent_dim,)) 
gan_output= discriminator(generator(gan_input)) 
gan= tf.keras.Model(gan_input, gan_output) 
gan.compile(optimizer=tf.keras.optimizers.Adam(0.0002), loss="binary_crossentropy") 
 
# Training the GAN 
def train_gan(generator, discriminator, gan, epochs, batch_size=128): 
    for epoch in range(epochs): 
        for _ in range(batch_size): 
            # Train discriminator 
            noise= tf.random.normal([batch_size, latent_dim]) 
            fake_images= generator.predict(noise) 
            real_images= x_train[np.random.randint(0, x_train.shape[0], batch_size)] 
             
            real_labels= tf.ones((batch_size, 1)) 
            fake_labels= tf.zeros((batch_size, 1)) 
             
            d_loss_real= discriminator.train_on_batch(real_images, real_labels) 
            d_loss_fake= discriminator.train_on_batch(fake_images, fake_labels) 
             
            # Train generator 
            misleading_labels= tf.ones((batch_size, 1)) 
            g_loss= gan.train_on_batch(noise, misleading_labels)
            print(f"Epoch {epoch + 1}/{epochs}, D Loss: {d_loss_real[0] + d_loss_fake[0]}, G Loss: {g_loss}") 
 
train_gan(generator, discriminator, gan, epochs=50) 

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step 


/home/angele/workspace/env/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1/50, D Loss: 1.420270562171936, G Loss: 0.6953308582305908
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

E0000 00:00:1739375158.787359   35621 meta_optimizer.cc:966] remapper failed: INVALID_ARGUMENT: Mutation::Apply error: fanout 'gradient_tape/functional_2_1/sequential_1_2/leaky_re_lu_1_2/LeakyRelu/LeakyReluGrad' exist for missing node 'functional_2_1/sequential_1_2/conv2d_1_2/BiasAdd'.


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Epoch 1/50, D Loss: 1.4126849174499512, G Loss: 0.6945578455924988
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch 1/50, D Loss: 1.411543369293213, G Loss: 0.6939259171485901
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Epoch 1/50, D Loss: 1.4111177921295166, G Loss: 0.6933797597885132
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Epoch 1/50, D Loss: 1.4115484952926636, G Loss: 0.6928658485412598
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Epoch 1/50, D Loss: 1.4117882251739502, G Loss: 0.6923456192016602
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch 1/50, D Loss: 1.412086009979248, G Loss: 0.6918238997459412
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Epoch 1/50, D Loss: 1.412508487701416, G Loss: 0.6913044452667236
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Epoch 1/50, D Loss: 1.4131124019622803, G Loss: 0.6907606720924377
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch 1/50, D Loss: 1.4134951829910278, G Loss: 0.6901665925979614
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Epoch 1/50, D Loss: 

def generate_images(generator, n_images): 
    noise = tf.random.normal([n_images, latent_dim]) 
    generated_images = generator.predict(noise) 
    fig, axes = plt.subplots(1, n_images, figsize=(20, 4)) 
    for i, img in enumerate(generated_images): 
        axes[i].imshow(img.squeeze(), cmap="gray") 
        axes[i].axis("off") 
    plt.show() 
 
generate_images(generator, 10)

In [2]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout 
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization 
from tensorflow.keras.models import Model 
import tensorflow as tf 
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Dense, Reshape, MultiHeadAttention, LayerNormalization


2025-02-12 18:26:56.462908: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-12 18:26:56.474006: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-12 18:26:56.554186: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-12 18:26:56.615627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739381216.676201    1379 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739381216.69

In [3]:
latent_dim= 100 
 
def transformer_generator(latent_dim):
 inputs = Input(shape=(latent_dim,))
 
 # Initial dense layer to project noise into a higher-dimensional space
 x = Dense(7 * 7 * 128, activation="relu")(inputs)
 x = Reshape((49, 128))(x) # Reshape to (7x7 patches, 128 features)
 # Positional Encoding
 position_encoding = tf.range(start=0, limit=49, delta=1)
 position_embedding = tf.keras.layers.Embedding(input_dim=49, 
output_dim=128)(position_encoding)
 x += position_embedding
 # Multi-Head Self-Attention
 x = MultiHeadAttention(num_heads=4, key_dim=128)(x, x)
 x = LayerNormalization()(x)
 # Feedforward network
 x = Dense(128, activation="relu")(x)
 x = LayerNormalization()(x)
# Reshape and upsample to image dimensions
 x = Reshape((7, 7, 128))(x)
 x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", 
activation="relu")(x)
 x = layers.Conv2DTranspose(1, kernel_size=4, strides=2, padding="same", 
activation="tanh")(x)
 model = Model(inputs, x, name="Transformer_Generator")
 return model
generator = transformer_generator(latent_dim)
generator.summary()


W0000 00:00:1739381224.272836    1379 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "Transformer_Generator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 6272)      │    633,472 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 49, 128)   │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 49, 128)   │          0 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 49, 128)   │    263,808 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 49, 128)   │        256 │ multi_head_atten… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 49, 128)   │     16,512 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 49, 128)   │        256 │ dense_1[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 7, 7, 128) │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 14, 14,    │    131,136 │ reshape_1[0][0]   │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 28, 28, 1) │      1,025 │ conv2d_transpose… │
│ (Conv2DTranspose)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,046,465 (3.99 MB)

 Trainable params: 1,046,465 (3.99 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Dense, Flatten, Reshape, MultiHeadAttention, LayerNormalization

def transformer_discriminator():
    inputs = Input(shape=(28, 28, 1))
    x = Flatten()(inputs)  
    x = Dense(49 * 128, activation="relu")(x) 
    x = Reshape((49, 128))(x)  # Now it correctly reshapes

    # Positional Encoding
    position_encoding = tf.range(start=0, limit=49, delta=1)
    position_embedding = layers.Embedding(input_dim=49, output_dim=128)(position_encoding)

    # Expand dimensions to match batch size
    position_embedding = tf.expand_dims(position_embedding, axis=0)  
    x = x + position_embedding  # Apply positional encoding

    # Multi-Head Self-Attention
    x = MultiHeadAttention(num_heads=4, key_dim=128)(x, x)
    x = LayerNormalization()(x)

    # Classification Layers
    x = Flatten()(x)  
    x = Dense(128, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)  # Binary classification

    model = Model(inputs, x, name="Transformer_Discriminator")
    return model

# Create and summarize the discriminator model
discriminator = transformer_discriminator()
discriminator.summary()


Model: "Transformer_Discriminator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 784)       │          0 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 6272)      │  4,923,520 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 49, 128)   │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 49, 128)   │          0 │ reshape_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 49, 128)   │    263,808 │ add_1[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 49, 128)   │        256 │ multi_head_atten… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 6272)      │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │    802,944 │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │        129 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,990,657 (22.85 MB)

 Trainable params: 5,990,657 (22.85 MB)

 Non-trainable params: 0 (0.00 B)